<a href="https://colab.research.google.com/github/chaufe/testcases/blob/main/colab/OpenRoad__invalid_pin_access_check_on_pads.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **Testcase to show OpenRoad to error-out on pin acces check on pad opening**

### Environment and Software Setup
- setup environment
- install conda-eda, GF180MCU PDK and EDA software
- install and patch helper software

In [ ]:
#@title Install a few packages required later-on { display-mode: "form" }
!dpkg --configure -a
!apt install python3-click

In [ ]:
#@title Install Anaconda environment {display-mode: "form"}
CONDA_PREFIX = '/content/eda'
%env CONDA_PREFIX = {CONDA_PREFIX}
!echo 'Will install CONDA EDA to {CONDA_PREFIX}'
!test -e Install.sh || wget -O Install.sh https://github.com/proppy/conda-eda/releases/download/v0.0-1445-gdbbed53/digital.gf180mcuc-0-Linux-x86_64.sh
!test -d {CONDA_PREFIX} || bash Install.sh -b -p {CONDA_PREFIX}

In [ ]:
#@title Update conda packages {display-mode: "form"}
!{CONDA_PREFIX}/bin/conda update --yes --all

In [ ]:
#@title Install EDA software { display-mode: "form" }

## required extra packages
#!{CONDA_PREFIX}/bin/conda install --yes libboost fmt==8.1.1 spdlog
#!{CONDA_PREFIX}/bin/conda install --yes litex-hub::tcllib

# use hard-coded packages as version resolution does not work as expected
#for pkg in [
#    'https://anaconda.org/LiteX-Hub/klayout/0.28.9_2_2_g508b52374/download/linux-64/klayout-0.28.9_2_2_g508b52374-20230606_125334_py37.tar.bz2',
#    'https://anaconda.org/LiteX-Hub/magic/8.3.404_0_g482d753/download/linux-64/magic-8.3.404_0_g482d753-20230606_125334.tar.bz2',
#    'https://anaconda.org/LiteX-Hub/netgen/1.5.254_0_g02d7a1b/download/linux-64/netgen-1.5.254_0_g02d7a1b-20230606_125334.tar.bz2',
#    'https://anaconda.org/LiteX-Hub/open_pdks.gf180mcuc/1.0.419_0_g3df14f8/download/noarch/open_pdks.gf180mcuc-1.0.419_0_g3df14f8-20230606_125334.tar.bz2',
#    'https://anaconda.org/LiteX-Hub/openlane/2023.04.12_0_g23e367a9/download/noarch/openlane-2023.04.12_0_g23e367a9-20230412_103222.tar.bz2',
#    'https://anaconda.org/LiteX-Hub/openroad/2.0_8931_g74ada9e7c/download/linux-64/openroad-2.0_8931_g74ada9e7c-20230606_125334.tar.bz2',
#    'https://anaconda.org/LiteX-Hub/yosys/0.30_17_g8b2a00102/download/linux-64/yosys-0.30_17_g8b2a00102-20230606_125334_py37.tar.bz2',
#    'https://repo.anaconda.com/pkgs/main/linux-64/ruby-3.1.4-h359c5ee_0.conda',
#    'https://repo.anaconda.com/pkgs/main/linux-64/python-3.7.16-h7a1cb2a_0.conda',
#    'https://anaconda.org/anaconda/click/8.0.4/download/linux-64/click-8.0.4-py37h06a4308_0.tar.bz2',
#]:
#  print("Installing " + pkg)
#  !{CONDA_PREFIX}/bin/conda install --yes {pkg}

# use hard-coded packages as version resolution does not work as expected
for pkg in [
    'https://anaconda.org/LiteX-Hub/openroad/2.0_8931_g74ada9e7c/download/linux-64/openroad-2.0_8931_g74ada9e7c-20230606_125334.tar.bz2',
]:
  print("Installing " + pkg)
  !{CONDA_PREFIX}/bin/conda install --yes {pkg}

In [ ]:
#@title Setup remaining variables { display-mode: 'form'}
%env CONDA_PREFIX = {CONDA_PREFIX}
import os
%env PATH = {CONDA_PREFIX + '/bin:' + os.getenv('PATH')}
%env PDK_ROOT = {CONDA_PREFIX + '/share/pdk'}
%env PDK = gf180mcuC
%env IO_LIB_PATH = {CONDA_PREFIX + '/share/pdk/gf180mcuC/libs.ref/gf180mcu_fd_io'}
%env SRAM_LIB_PATH = {CONDA_PREFIX + '/share/pdk/gf180mcuC/libs.ref/gf180mcu_fd_ip_sram'}
!echo $PYTHONPATH

In [ ]:
#@title Create patch file for GF180MCU IO IP { display-mode: "form" }
#!wget https://raw.githubusercontent.com/chaufe/testcases/main/patches/gf180mcu_fd_io.lef.patch
%%writefile "gf180mcu_fd_io.lef.patch"
--- eda/share/pdk/gf180mcuC/libs.ref/gf180mcu_fd_io/lef/gf180mcu_fd_io.lef	2023-02-13 13:43:43.000000000 +0100
+++ eda/share/pdk/gf180mcuC/libs.ref/gf180mcu_fd_io/lef/gf180mcu_fd_io.lef	2023-03-30 10:19:34.792862000 +0200
@@ -3,6 +3,20 @@
   DIVIDERCHAR "/" ;
   BUSBITCHARS "[]" ;
+
+SITE GF_IO_Site
+  SYMMETRY X Y ;
+  CLASS PAD ;
+  SIZE 1 BY 350 ;
+END GF_IO_Site
+
+SITE GF_COR_Site
+  SYMMETRY X Y ;
+  CLASS PAD ;
+  SIZE 355 BY 355 ;
+END GF_COR_Site
+
+
 MACRO gf180mcu_fd_io__asig_5p0
-  CLASS PAD INOUT ;
+  CLASS PAD AREAIO ;
   FOREIGN gf180mcu_fd_io__asig_5p0 ;
   ORIGIN 0.000 0.000 ;
@@ -815,5 +829,5 @@

 MACRO gf180mcu_fd_io__bi_24t
-  CLASS PAD INOUT ;
+  CLASS PAD AREAIO ;
   FOREIGN gf180mcu_fd_io__bi_24t ;
   ORIGIN 0.000 0.000 ;
@@ -1669,5 +1683,5 @@

 MACRO gf180mcu_fd_io__bi_t
-  CLASS PAD INOUT ;
+  CLASS PAD AREAIO ;
   FOREIGN gf180mcu_fd_io__bi_t ;
   ORIGIN 0.000 0.000 ;
@@ -3807,5 +3821,5 @@

 MACRO gf180mcu_fd_io__dvdd
-  CLASS PAD POWER ;
+  CLASS PAD AREAIO ;
   FOREIGN gf180mcu_fd_io__dvdd ;
   ORIGIN 0.000 0.000 ;
@@ -4600,5 +4614,5 @@

 MACRO gf180mcu_fd_io__dvss
-  CLASS PAD POWER ;
+  CLASS PAD AREAIO ;
   FOREIGN gf180mcu_fd_io__dvss ;
   ORIGIN 0.000 0.000 ;
@@ -7609,5 +7623,5 @@

 MACRO gf180mcu_fd_io__in_c
-  CLASS PAD INPUT ;
+  CLASS PAD AREAIO ;
   FOREIGN gf180mcu_fd_io__in_c ;
   ORIGIN 0.000 0.000 ;
@@ -8416,5 +8430,5 @@

 MACRO gf180mcu_fd_io__in_s
-  CLASS PAD INPUT ;
+  CLASS PAD AREAIO ;
   FOREIGN gf180mcu_fd_io__in_s ;
   ORIGIN 0.000 0.000 ;


In [ ]:
#@title Patch GF180MCU IP { display-mode: "form" }
!patch -p0 < gf180mcu_fd_io.lef.patch

### Create Testcase

In [8]:
#@title Write testcase RTL { display-mode: "form" }
%%writefile "testcase.v"
module testcase (input clk,
                 input din,
                 output wire dout);

   wire clk_int;
   wire din_int;
   reg  dout_int;


   // pads
   (* keep *) gf180mcu_fd_io__bi_t pad_dout(
                                            .PU(1'b0), .PD(1'b0),
                                            .OE(1'b1), .IE(1'b0),
                                            .PDRV0(1'b0), .PDRV1(1'b0),
                                            .CS(1'b0), .SL(1'b0),
                                            .PAD(dout),
                                            .A(dout_int),
                                            .Y()
                                            );

   (* keep *) gf180mcu_fd_io__in_s pad_clk(
                                           .PU(1'b0), .PD(1'b0),
                                           .PAD(clk),
                                           .Y(clk_int)
                                           );

   (* keep *) gf180mcu_fd_io__in_s pad_din(
                                           .PU(1'b0), .PD(1'b0),
                                           .PAD(din),
                                           .Y(din_int)
                                           );

    always @(posedge clk_int)
      dout_int <= din_int;

endmodule


Writing testcase.v


In [9]:
#@title Write config.tcl to control OpenLANE { display-mode: "form" }
%%writefile "config.tcl"

set ::env(DESIGN_NAME) "testcase"

set ::env(VERILOG_FILES) [list "testcase.v" ]

set ::env(EXTRA_LIBS) [list \
  $::env(IO_LIB_PATH)/liberty/gf180mcu_fd_io__tt_025C_5v00.lib \
  $::env(IO_LIB_PATH)/liberty/gf180mcu_fd_io__ff_n40C_5v50.lib \
  $::env(IO_LIB_PATH)/liberty/gf180mcu_fd_io__ff_125C_5v50.lib \
  $::env(IO_LIB_PATH)/liberty/gf180mcu_fd_io__ss_125C_4v50.lib \
]

set ::env(EXTRA_LEFS) [list \
]

#set ::env(BASE_SDC_FILE) "testcase.sdc"

set ::env(EXTRA_GDS_FILES) [list \
  $::env(IO_LIB_PATH)/gds/gf180mcu_fd_io.gds \
]

set ::env(DESIGN_IS_CORE) 1
set ::env(FP_PDN_CORE_RING) 1

set ::env(GRT_ALLOW_CONGESTION) 1
set ::env(DRT_OPT_ITERS) 10

set ::env(DIODE_INSERTION_STRATEGY) 0

set ::env(USE_GPIO_PADS) 1
set ::env(GPIO_PADS_LEF) $::env(IO_LIB_PATH)/lef/gf180mcu_fd_io.lef
unset ::env(GPIO_PADS_VERILOG)
set ::env(FP_IO_MODE) 1

set ::env(FP_PDN_UPPER_LAYER) Metal5
set ::env(FP_PDN_LOWER_LAYER) Metal4
set ::env(FP_PDN_RAILS_LAYER) Metal1
set ::env(FP_PDN_CORE_RING_VWIDTH) 5.0
set ::env(FP_PDN_CORE_RING_HWIDTH) 5.0
set ::env(FP_PDN_CORE_RING_VSPACING) 1.0
set ::env(FP_PDN_CORE_RING_HSPACING) 1.0

set ::env(SYNTH_NO_FLAT) 0
set ::env(SYNTH_SHARE_RESOURCES) 1

set ::env(CLOCK_TREE_SYNTH) 1
set ::env(CLOCK_PORT) "clk"
set ::env(CLOCK_PERIOD) 100

set ::env(FP_SIZING) absolute
set ::env(DIE_AREA) "0 0 1460 1460"
set ::env(CORE_AREA) "370 370 1090 1090"

set ::env(PL_TARGET_DENSITY) 0.65

# MAGIC GDS-out controls
## ... try GDS with hierarchy
set ::env(MAGIC_DISABLE_HIER_GDS) 0
set ::env(MAGIC_GENERATE_LEF) 0
set ::env(MAGIC_GENERATE_MAGLEF) 0

Writing config.tcl


In [10]:
#@title Use updated ioplace script to create pad ring { display-mode: "form" }
%%writefile "eda/share/openlane/scripts/openroad/ioplacer.tcl"
# Copyright 2020-2022 Efabless Corporation
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#      http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
source $::env(SCRIPTS_DIR)/openroad/common/io.tcl
read

if { [info exists ::env(CONTEXTUAL_IO_FLAG)] } {
	read_lef $::env(placement_tmpfiles)/top_level.lef
}

if {$::env(FP_IO_HLENGTH) != "" && $::env(FP_IO_HLENGTH) != ""} {
	set_pin_length -hor_length $::env(FP_IO_HLENGTH) \
		-ver_length $::env(FP_IO_VLENGTH)
}

if {$::env(FP_IO_HEXTEND) != "0" && $::env(FP_IO_VEXTEND) != "0"} {
	set_pin_length_extension -hor_extension $::env(FP_IO_HEXTEND) \
		-ver_extension $::env(FP_IO_VEXTEND)
}

if {$::env(FP_IO_VTHICKNESS_MULT) != "" && $::env(FP_IO_HTHICKNESS_MULT) != ""} {
	set_pin_thick_multiplier -hor_multiplier $::env(FP_IO_HTHICKNESS_MULT) \
		-ver_multiplier $::env(FP_IO_VTHICKNESS_MULT)
}

set arg_list [list]
if { $::env(FP_IO_MODE) == 1 } {
	lappend arg_list -random
}

if { $::env(FP_IO_MIN_DISTANCE) != "" } {
	lappend arg_list -min_distance $::env(FP_IO_MIN_DISTANCE)
}

set HMETAL $::env(FP_IO_HLAYER)
set VMETAL $::env(FP_IO_VLAYER)

place_pins {*}$arg_list \
	-random_seed 42 \
	-hor_layers $HMETAL \
	-ver_layers $VMETAL

# create io-ring
make_io_sites -horizontal_site GF_IO_Site -vertical_site GF_IO_Site -corner_site GF_COR_Site -offset 0
place_pad -master gf180mcu_fd_io__dvdd   -row IO_NORTH -location  400.0 pad_dvdd1
place_pad -master gf180mcu_fd_io__in_s   -row IO_NORTH -location  700.0 pad_clk
place_pad -master gf180mcu_fd_io__dvss   -row IO_NORTH -location 1000.0 pad_dvss1

place_pad -master gf180mcu_fd_io__dvdd   -row IO_EAST  -location  400.0 pad_dvdd2
place_pad -master gf180mcu_fd_io__in_s   -row IO_EAST  -location  700.0 pad_din
place_pad -master gf180mcu_fd_io__dvss   -row IO_EAST  -location 1000.0 pad_dvss2

place_pad -master gf180mcu_fd_io__dvdd   -row IO_WEST  -location  400.0 pad_dvdd3
place_pad -master gf180mcu_fd_io__bi_24t -row IO_WEST  -location  700.0 pad_dout
place_pad -master gf180mcu_fd_io__dvss   -row IO_WEST  -location 1000.0 pad_dvss3

place_pad -master gf180mcu_fd_io__dvdd   -row IO_SOUTH -location  400.0 pad_dvdd4
place_pad -master gf180mcu_fd_io__dvss   -row IO_SOUTH -location 1000.0 pad_dvss4

place_corners gf180mcu_fd_io__cor
foreach io_row { IO_EAST IO_WEST IO_SOUTH IO_NORTH } {
    place_io_fill -row $io_row gf180mcu_fd_io__fill10 gf180mcu_fd_io__fill5 gf180mcu_fd_io__fill1
}

connect_by_abutment

# work-around for pin-access error message
if {[info exists ::env(WORKAROUND)] && ($env(WORKAROUND) eq "enabled") } {

  # get db/chip/block
  set db [ord::get_db]
  set chip [$db getChip]
  set block [$chip getBlock]
  set dbu_m [$block getDbUnitsPerMicron]

  # mark all nets connected to pins (terminals) as special-net so OpenROAD will not try to route it
  foreach bterm [$block getBTerms] {
    if {$bterm eq "NULL"} {
        error "Could not get terminals of block"
    } else {
      set net [$bterm getNet]
      if {$net eq "NULL"} {
        error "Block terminal [$bterm getName] returned NULL net"
      } else {
        puts "Setting net [$net getName] (connected to block terminal [$bterm getName]) to special-net"
        $net setSpecial
      }
    }
  }
}


write


Overwriting eda/share/openlane/scripts/openroad/ioplacer.tcl


### Run Testcase

In [11]:
#@title Enable or disable work-around { display-mode: "form" }
import ipywidgets as widgets
import IPython

workaround = widgets.RadioButtons(
    options=['enabled', 'disabled'],
    value='disabled',
    disabled=False
)
print('Enable or disable work-around?:')
IPython.display.display(workaround)

Enable or disable work-around?:


RadioButtons(index=1, options=('enabled', 'disabled'), value='disabled')

In [ ]:
#@title Run OpenLANE { display-mode: "form" }
%env WORKAROUND = {workaround.value}
%env PYTHONPATH = /usr/lib/python310.zip;/usr/lib/python3.10:/usr/lib/python3.10/lib-dynload:/usr/local/lib/python3.10/dist-packages:/usr/lib/python3/dist-packages:/usr/local/lib/python3.10/dist-packages/IPython/extensions
!flow.tcl -design . -to routing

In [ ]:
#@title Package issue { display-mode: "form" }
import pathlib
STEP='25'
odb = sorted(pathlib.Path('/content/runs/').glob(f'*/tmp/*/{STEP}-*.odb'))[-1].resolve()

%env PYTHONPATH = /content/eda/lib/python3.7:/content/eda/lib/python37.zip:/content/eda/lib/python3.7/lib-dynload:/content/eda/lib/python3.7/site-packages
!python3.7 /content/eda/share/openlane/scripts/or_issue.py\
    --tool openroad\
    --script /content/eda/share/openlane/scripts/openroad/droute.tcl {odb}

!tar czvf _build.tgz _build
